In [2]:
# for each dataset dir, open 'test_results.csv', and for each column calculate the mean and std of the values (except the first column)
# then print the results in a table
import pickle as pkl
import os

In [3]:
labels_path = '/data/home/gabrielg/Bounded_Future_from_GIT/data/MultiBypass140/transcriptions'
splits_path = '/data/home/gabrielg/Bounded_Future_from_GIT/data/MultiBypass140/Splits'
bern_path = os.path.join(labels_path, 'BernBypass70')
stras_path = os.path.join(labels_path, 'StrasBypass70')

In [4]:
splits_to_write = {'val':   {'0':[], '1':[], '2':[], '3':[], '4':[]}, 
                   'train': {'0':[], '1':[], '2':[], '3':[], '4':[]}, 
                   'test':  {'0':[], '1':[], '2':[], '3':[], '4':[]}}
for main_dir in [bern_path, stras_path]:
    # print(main_dir)
    for dir in os.listdir(main_dir):
        # print(dir)
        for file in os.listdir(os.path.join(main_dir, dir)):
            # get current split
            cur_split = file.split('.')[0].split('_')[-1]
            # write labels to transcriptions files
            with open(os.path.join(main_dir, dir, file), 'rb') as f:
                labels = pkl.load(f)
            for vid in labels.keys():
                with open(os.path.join(labels_path, vid + '.txt'), 'w') as f:
                    cur_step = 99 # doesn't exists in the dataset
                    for frame in labels[vid]:
                        end_frame = frame['Original_frame_id']
                        # if it's a new phase
                        if (cur_step != frame['Step_gt']):
                            # 1st time
                            if(cur_step != 99):
                                f.write(str(start_frame) + ' ' + str(end_frame) + ' S' + str(cur_step) + '\n')
                            cur_step = frame['Step_gt']
                            start_frame = frame['Original_frame_id']+1
                        # if it's the last frame
                        if end_frame == labels[vid][-1]['Original_frame_id']:
                            f.write(str(start_frame) + ' ' + str(end_frame+1) + ' S' + str(cur_step) + '\n')
                            break
                if dir in ['val', 'train', 'test']:
                    splits_to_write[dir][cur_split].append((vid, end_frame+1))
                else:
                    continue

In [7]:
# Assuming splits_to_write is structured as {dir: {split: [(vid, end_frame+1), ...]}}
for dir in ['val', 'train', 'test']:
    for cur_split in splits_to_write[dir]:
        # Sort the list of tuples for the current split
        sorted_data = sorted(splits_to_write[dir][cur_split], key=lambda x: x[0])  # Sorting by vid here

        # Determine the file name
        file_name = f'data_{dir}_{cur_split}.csv'
        
        # Write the sorted data to the file
        with open(os.path.join(splits_path, file_name), 'w') as f:
            for vid, end_frame in sorted_data:
                f.write(f'{vid},{end_frame}\n')